In [1]:
.include header

In [2]:
%%file header/simple-piecewise-polynomial.h

#pragma once

#include "polynomial.h"
#include "eliastocco_namespace_piecewise-polynomial.h"
#include "square_range.h"

namespace eliastocco {

    namespace polynomial {

        template < class type , t_dim dimension = 1  , t_dim codimension = 1 >
                class simple_piecewise_polynomial :
                    public polynomial<type,dimension,codimension> {

                    #define thisclass simple_piecewise_polynomial<type,dimension,codimension>
                    #define motherclass polynomial<type,dimension,codimension>
                    
                    using motherclass :: polynomial;
                    
                   public : 
                        typedef typename motherclass :: Type_out Type_out;
                        typedef typename motherclass :: Type_in Type_in;
                
//********************************************************************************************************************

                    //constructor
                    public: 
                            thisclass ( ) 
                                : domain ( [](auto x){return true;} ),
                                    motherclass() {};
                                    
                    //constructor
                    public: 
                            thisclass ( const motherclass pol ) 
                                : domain ( [](auto x){return true;} ),
                                    motherclass(pol) {};
                    
                    //constructor
                    public :
                        thisclass ( basis<type,dimension> * user_basis_function )
                            : thisclass ( motherclass ( user_basis_function ) ){};
                                            
                                            
                    //constructor
                    public: 
                            thisclass ( domain_t<type,dimension> user_domain , const motherclass pol ) 
                                : domain ( user_domain ),
                                    motherclass(pol) {};
                    
//********************************************************************************************************************

                    //valutazione del polinomio in x
                    public :
                         Type_out operator () ( const Type_in x ) const {
                         
                             if ( domain(x) ){   
                                 auto m = static_cast<motherclass>(*this);
                                 return m.operator()(x);
                             } else {
                                 return Type_out();
                             }
                        };
                        
                    //vectorized polinomial evaluation
                    public :
                    template < template < class ... > class vect = std :: vector >
                        vect <Type_out> operator () ( const vect < Type_in > x ) const { 
                            vect <Type_out> y (x.size());
                            std :: transform( std :: begin(x), std :: end(x), std :: begin(y),
                                [&]( auto a ) {return (*this)(a);});   
                            return y;

                        }; 

//********************************************************************************************************************
                    
                    //piccola ottimizzazione nel caso si moltiplichi per un polinomio non definito a tratti
                    //in realtà la presenza di questo operatore è necessaria per non creare confusione
                    //tra l'operator* definito qui sotto e quello definito in polynomial.h
                    public : 
                        friend thisclass operator * 
                            (   const thisclass left ,
                                const motherclass right ) {                                
                                thisclass output = static_cast<motherclass>(left) * right;                            
                                output.set_domain( left.get_domain() );                                
                                return output;                                
                            };
                            
                     //piccola comodià: l'operatore è commutativo     
                     public : 
                        friend thisclass operator * 
                            (   const motherclass left ,
                                const thisclass right ) {   
                                return right*left;                                
                            };
                            
                     public :
                        friend thisclass operator * 
                            (   const thisclass left ,
                                const thisclass right ) {
                                
                                thisclass output = static_cast<motherclass>(left) * 
                                                   static_cast<motherclass>(right); 

                                auto intersection_range = [=]( const t_var<type,dimension> x ){         
                                    const auto left_domain  = left.get_domain();
                                    const auto right_domain = right.get_domain();                                    
                                    return left_domain(x)*right_domain(x);   
                                };
                                
                                output.set_domain( intersection_range );
                                
                                return output;
                                
                            };                            
                  
//********************************************************************************************************************

                    //using motherclass :: operator*(const type factor , const thisclass right);
                    // moltiplicazione per uno scalare
                    public :
                        friend thisclass operator * ( const type factor , const thisclass right ){                        
                            
                            motherclass mother = factor * static_cast<motherclass>(right);                            
                            thisclass output (right.get_domain(),mother);                                
                            return output;                            
                            
                        };
                        
//********************************************************************************************************************
                    
                    public :
                        friend thisclass operator + 
                            (   const thisclass left ,
                                const thisclass right ) = delete ;
                                
                    // somma tra due polinomi: non è definibile!
                        
//********************************************************************************************************************
                    public :
                        void set_domain ( domain_t<type,dimension> user_domain ) {   
                            domain=user_domain ; 
                        };
                        
                    public :
                        void set_domain ( const square_range<type,dimension> user_domain ) {   
                            domain=user_domain.lambda ; 
                        };
                        
                    public :
                        inline auto get_domain ( ) const { return domain ; };

//********************************************************************************************************************

                    private :
                        domain_t<type,dimension> domain;
                        
                    #undef thisclass
                    #undef motherclass
                    
                    };
                
        
    }
}

Overwriting header/simple-piecewise-polynomial.h


In [3]:
#include "simple-piecewise-polynomial.h"

In [4]:
using namespace eliastocco :: polynomial ;

In [5]:
auto base = monomial_basis<double,1>(5)

In [6]:
basis_function_default <double,1> = monomial_basis<double>(5)

@0x7f069a270068

In [7]:
simple_piecewise_polynomial <double> pp

polynomial :: constructor


In [8]:
simple_piecewise_polynomial <double> pP (&base)

polynomial :: constructor


In [9]:
pp.set(0,1)

true

In [10]:
pp({1.0,2.0,4.5})

{ 1.0000000, 1.0000000, 1.0000000 }

In [11]:
pP.set(0,1);
pP({1.0,2.0,4.5})

{ 1.0000000, 1.0000000, 1.0000000 }

In [10]:
pp.set(2,4)

true

In [11]:
auto d = square_range<double,1>(0.0,1.0)

In [12]:
pp.set_domain(d)

In [13]:
pp({-1.0,-0.5,0.0,0.5,1.0,1.5})

{ 0.0000000, 0.0000000, 1.0000000, 2.0000000, 0.0000000, 0.0000000 }

In [14]:
simple_piecewise_polynomial <double> p1(&base);
simple_piecewise_polynomial <double> p2(&base);

polynomial :: constructor
polynomial :: constructor


In [15]:
p1.set_domain(square_range<double,1>(0.0,1.0));
p2.set_domain(square_range<double,1>(0.5,4.0));

In [16]:
p1.set(0,2);
p2.set(0,4);

In [17]:
auto p3 = p1*p2

polynomial :: constructor


In [18]:
p1({-1.0,-0.5,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5})

{ 0.0000000, 0.0000000, 2.0000000, 2.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }

In [19]:
p2({-1.0,-0.5,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5})

{ 0.0000000, 0.0000000, 0.0000000, 4.0000000, 4.0000000, 4.0000000, 4.0000000, 4.0000000, 4.0000000, 4.0000000, 0.0000000, 0.0000000 }

In [20]:
p3({-1.0,-0.5,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5})

{ 0.0000000, 0.0000000, 0.0000000, 8.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }

In [21]:
auto sum_p = 3.0*p1

In [22]:
sum_p({-1.0,-0.5,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5})

{ 0.0000000, 0.0000000, 6.0000000, 6.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000, 0.0000000 }

In [23]:
p1*p1

polynomial :: constructor


@0x7fffcd15f010

In [24]:
p2*p2

polynomial :: constructor


@0x7fffcd607850

In [25]:
polynomial<double,1> pol

In [26]:
p1*pol

@0x7fffd8652790

In [27]:
pol*p1

@0x7fffd84bd4b0